In [1]:
import kss
from typing import List, Dict
import zipfile
from pathlib import Path
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb

# pd.set_option('display.max_columns', None)      # 출력할 최대 컬럼 수 제한 해제            # 터미널 폭에 맞추지 않음
# pd.set_option('display.max_colwidth', None)
# 원본 파일과 임시 복구 파일 경로
orig = Path.cwd() / "건축 데이터.xlsx"
fixed = Path.cwd() / "건축_데이터_fixed.xlsx"

# 1) ZIP 내부를 순회하며 styles.xml만 건너뛰고 재압축
with zipfile.ZipFile(orig, 'r') as zin, zipfile.ZipFile(fixed, 'w') as zout:
    for item in zin.infolist():
        if item.filename == "xl/styles.xml":
            # styles.xml을 제거
            continue
        data = zin.read(item.filename)
        zout.writestr(item, data)

# 2) 복구된 파일 읽기
df = pd.read_excel(
    fixed,
    sheet_name="Page",
    engine="openpyxl"  # 이제는 정상적으로 로드될 것입니다.
)

# ex_df = pd.read_excel(
#     fixed,
#     sheet_name="data",
#     engine="openpyxl"  # 이제는 정상적으로 로드될 것입니다.
# )
model = SentenceTransformer("dragonkue/snowflake-arctic-embed-l-v2.0-ko")

/Users/gim-yonghyeon/Documents/GitHub/image-search/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 사용할 컬럼 인덱스
cols = [0,1,3,4,5,6,8,9,10,12,14,15,17,19,20,21,22,23,24,25,26]

# 1) 3행의 NaN은 4행 값으로 채워서 헤더 이름 획득
header = df.iloc[3, cols].fillna(df.iloc[4, cols]).tolist()

# 2) 바로 위(2행)의 값 (여기선 '설계자 답변', '검증위원 평가' 등) 준비
parent = df.iloc[2, cols].astype(str).tolist()

# 3) 고유 이름 생성
counts = {}
unique_names = []

for i, name in enumerate(header):
    # ① 만약 '내용' 컬럼이면, 이전 이름 뒤에 ' 내용' 추가
    if name == "내용":
        if unique_names:
            unique_names.append(f"{unique_names[-1]} 내용")
        else:
            # 만약 첫 컬럼이 '내용'이라면 parent로 대체
            unique_names.append(f"{parent[i]} 내용")
        continue

    # ② 일반 컬럼명 중복 처리: 두 번째부터 _1, _2 …
    counts[name] = counts.get(name, 0) + 1
    if counts[name] == 1:
        unique_names.append(name)
    else:
        unique_names.append(f"{name}_{counts[name]-1}")

# print(unique_names)

fix_data = df.iloc[5:,cols].reset_index(drop=True)

fix_data.columns = unique_names

fix_data = fix_data.iloc[:-1, :]

In [3]:
def chunk_with_sliding_window(text: str,
                              max_sentences: int = 5,
                              overlap: int = 1) -> List[str]:
    sentences = kss.split_sentences(text)
    if len(sentences) <= max_sentences:
        return [" ".join(sentences)]
    chunks = []
    step = max_sentences - overlap
    for start in range(0, len(sentences), step):
        chunks.append(" ".join(sentences[start:start+max_sentences]))
        if start + max_sentences >= len(sentences):
            break
    return chunks

fix_data["combined_text"] = (
    fix_data
    .astype(str)
    .where(~fix_data.isna(), "")
    .agg("\n".join, axis=1)
)

# (기존) fix_data["combined_text"] 생성
documents = fix_data["combined_text"].tolist()
all_metadatas = fix_data.drop(columns="combined_text").to_dict(orient="records")
original_ids = df.index[5:].tolist()[:-1]  # 마지막 행 제거 고려

chunked_texts, chunked_metas, chunked_ids = [], [], []
for doc, meta, oid in zip(documents, all_metadatas, original_ids):
    for idx, chunk in enumerate(chunk_with_sliding_window(doc, 5, 1)):
        chunked_texts.append(chunk)
        chunked_metas.append(meta)
        chunked_ids.append(f"{oid}_{idx}")

# 임베딩 및 저장
embeddings = model.encode(chunked_texts,
                          show_progress_bar=True,
                          convert_to_numpy=True).tolist()
client = chromadb.PersistentClient(path="./chroma_db")
col = client.get_or_create_collection(name="fix_const")
col.add(
    documents=chunked_texts,
    embeddings=embeddings,
    metadatas=chunked_metas,
    ids=chunked_ids
)

[Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class

/Users/gim-yonghyeon/Documents/GitHub/image-search/.venv/lib/python3.10/site-packages/pecab/_tokenizer.py:265: RuntimeWarning: overflow encountered in scalar add
  from_pos_data.costs[idx]
/Users/gim-yonghyeon/Documents/GitHub/image-search/.venv/lib/python3.10/site-packages/pecab/_tokenizer.py:274: RuntimeWarning: overflow encountered in scalar add
  least_cost += word_cost
Batches: 100%|██████████████████████████████████| 16/16 [00:43<00:00,  2.75s/it]
